# Exemplo 15: Varejo - Monitoramento de Estoque (Spark)

Este notebook demonstra o monitoramento de estoque em tempo real baseado em logs de vendas usando **Spark Streaming**.

**Cenário**: Baixar o estoque conforme as vendas ocorrem e alertar quando o nível estiver crítico (< 10 unidades).

## 1. Configuração

In [ ]:
# Instalar Java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Baixar e Instalar Spark
!wget https://archive.apache.org/dist/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz && tar xf spark-3.5.0-bin-hadoop3.tgz

# Baixar e Instalar Kafka
!wget https://archive.apache.org/dist/kafka/3.6.1/kafka_2.13-3.6.1.tgz && tar xf kafka_2.13-3.6.1.tgz

# Instalar pacotes Python
!pip install -q findspark pyspark kafka-python

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"
import findspark
findspark.init()

## 2. Iniciar Kafka

In [ ]:
%%bash
cd kafka_2.13-3.6.1
bin/zookeeper-server-start.sh -daemon config/zookeeper.properties
sleep 5
bin/kafka-server-start.sh -daemon config/server.properties
sleep 5
bin/kafka-topics.sh --create --topic retail-sales --bootstrap-server localhost:9092 --replication-factor 1 --partitions 1

## 3. Simulador de Vendas

In [ ]:
import json
import time
import random
from kafka import KafkaProducer
import threading

def generate_sales():
    producer = KafkaProducer(bootstrap_servers=['localhost:9092'],
                             value_serializer=lambda x: json.dumps(x).encode('utf-8'))
    products = ['PROD-A', 'PROD-B', 'PROD-C']
    
    try:
        for _ in range(500):
            data = {'product_id': random.choice(products), 'quantity': random.randint(1, 5), 'timestamp': time.time()}
            producer.send('retail-sales', value=data)
            time.sleep(0.1)
    finally:
        producer.close()

thread = threading.Thread(target=generate_sales)
thread.start()

## 4. Monitoramento de Estoque

In [ ]:
%%writefile kafka_consumer.py
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col, sum as _sum
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType

spark = SparkSession.builder.appName("InventoryMonitor").getOrCreate()

schema = StructType([
    StructField("product_id", StringType()),
    StructField("quantity", IntegerType()),
    StructField("timestamp", FloatType())
])

df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "retail-sales") \
    .load()

sales = df.select(from_json(col("value").cast("string"), schema).alias("data")).select("data.*")

# Calcular vendas totais (saída de estoque)
total_sales = sales.groupBy("product_id").agg(_sum("quantity").alias("total_sold"))

# Simular Estoque Inicial como constante (numa app real viria de um DB)
initial_stock = 300

inventory_status = total_sales.withColumn("current_stock", initial_stock - col("total_sold"))

# Alertar se estoque < 50
# (Usando 250 vendidos como limiar para o exemplo rodar rápido)
alerts = inventory_status.filter(col("current_stock") < 250)

query = alerts.writeStream \
    .outputMode("complete") \
    .format("console") \
    .start()


query.awaitTermination()

In [ ]:
!spark-submit --packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0 kafka_consumer.py